In [1]:
from IPython.display import display

from io import StringIO
import itertools
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd

from mosek.fusion import Model, Domain, Expr, ObjectiveSense, SolutionStatus


In [2]:
# supply = pd.read_csv('20201007_da_co.processed.csv').set_index('id')
supply = pd.DataFrame([
    ['Bus 1', 100., 12.],
    ['Bus 2', 80., 20.],
], columns = ['node', 'capacity (MW)', 'offer ($/MW)'])
nsupply = len(supply)
print(supply.shape)
supply.head()

(2, 3)


,node,capacity (MW),offer ($/MW)
0,Bus 1,100.0,12.0
1,Bus 2,80.0,20.0


In [3]:
# demand = pd.read_csv('20201007_bids_cb.processed.csv').set_index('id')
demand = pd.DataFrame([
    ['Bus 2', 100., 40.],
    ['Bus 3', 50., 35.],
], columns = ['node', 'demand (MW)', 'bid ($/MW)'])
ndemand = len(demand)
print(demand.shape)
demand.head()

(2, 3)


,node,demand (MW),bid ($/MW)
0,Bus 2,100.0,40.0
1,Bus 3,50.0,35.0


In [4]:
nodes = sorted(set(supply['node']) | set(demand['node']))
nnodes = len(nodes)
print(nodes)

['Bus 1', 'Bus 2', 'Bus 3']


In [5]:
demand.groupby(demand['bid ($/MW)'] >= 2000.)['node'].count()

bid ($/MW)
False    2
Name: node, dtype: int64

In [6]:
lines = pd.DataFrame([
    ['%s-%s' % (src, dest), src, dest, 100., 500.] 
    for src, dest in itertools.combinations(nodes, 2)
], columns=['id', 'source', 'dest', 'capacity (MW)', 'susceptance (S)']).set_index('id')
nlines = len(lines)
lines

,source,dest,capacity (MW),susceptance (S)
id,,,,
Bus 1-Bus 2,Bus 1,Bus 2,100.0,500.0
Bus 1-Bus 3,Bus 1,Bus 3,100.0,500.0
Bus 2-Bus 3,Bus 2,Bus 3,100.0,500.0


In [16]:
rev_lines = lines.copy()
rev_lines.source = lines.dest
rev_lines.dest = lines.source
bi_lines = pd.concat([lines, rev_lines]).set_index(['source', 'dest'], drop=False)
n_bi_lines = len(bi_lines)
bi_lines


source   dest  capacity (MW)  susceptance (S)
source dest                                               
Bus 1  Bus 2  Bus 1  Bus 2          100.0            500.0
       Bus 3  Bus 1  Bus 3          100.0            500.0
Bus 2  Bus 3  Bus 2  Bus 3          100.0            500.0
       Bus 1  Bus 2  Bus 1          100.0            500.0
Bus 3  Bus 1  Bus 3  Bus 1          100.0            500.0
       Bus 2  Bus 3  Bus 2          100.0            500.0

In [8]:
pd.concat([demand.groupby('node')[['demand (MW)']].sum(),
           supply.groupby('node')[['capacity (MW)']].sum()], axis='columns')

,demand (MW),capacity (MW)
node,,
Bus 2,100.0,80.0
Bus 3,50.0,NaN
Bus 1,NaN,100.0


In [19]:
M = Model('dual')

log = StringIO()
M.setLogHandler(log)

#dual variables
muDup = M.variable('muDup', ndemand, Domain.greaterThan(0.))
muDdown = M.variable('muDdown', ndemand, Domain.greaterThan(0.))
muGup = M.variable('muGup', nsupply, Domain.greaterThan(0.))
muGdown = M.variable('muGdown', nsupply, Domain.greaterThan(0.))
etaup = M.variable('etaup', n_bi_lines, Domain.greaterThan(0.))
etadown = M.variable('etadown', n_bi_lines, Domain.greaterThan(0.))
lambdaN = M.variable('lambdaN', nnodes, Domain.unbounded())
gamma = M.variable('gamma', 1, Domain.unbounded())

## first order conditions

#dL/dpD
demand_idx = [nodes.index(x) for x in demand.node]
dldpd = Expr.sub(muDup, demand['bid ($/MW)'].values)
dldpd = Expr.sub(dldpd, muDdown)
dldpd = Expr.add(dldpd, lambdaN.pick(demand_idx))
pd_const = M.constraint('dldpd', dldpd, Domain.equalsTo(0.))

#dL/dpG
supply_idx = [nodes.index(x) for x in supply.node]
dldpg = Expr.add(muGup, supply['offer ($/MW)'].values)
dldpg = Expr.sub(dldpg, muGdown)
dldpg = Expr.sub(dldpg, lambdaN.pick(supply_idx))
pg_const = M.constraint('dldpg', dldpg, Domain.equalsTo(0.))

theta_eqs = []
for n, n_node in enumerate(nodes):
    if n == 0:
        dldtheta = gamma
    else:
        dldtheta = Expr.zeros(1)
    for m_node in bi_lines.xs(n_node).index:
        m = nodes.index(m_node)

        n_m_idx = bi_lines.index.get_loc((n_node, m_node))
        m_n_idx = bi_lines.index.get_loc((m_node, n_node))
        nm_duals = Expr.sub(etaup.index(n_m_idx), 
                            etaup.index(m_n_idx))
        nm_duals = Expr.sub(nm_duals, etadown.index(n_m_idx))
        nm_duals = Expr.add(nm_duals, etadown.index(m_n_idx))
        nm_duals = Expr.add(nm_duals, lambdaN.index(n))
        nm_duals = Expr.sub(nm_duals, lambdaN.index(m))
        dldtheta = Expr.add(
            dldtheta, 
            Expr.mul(bi_lines.loc[(n_node, m_node), 'susceptance (S)'],
                     nm_duals))
    theta_eqs.append(
        M.constraint('dldtheta%d' % n, dldtheta, Domain.equalsTo(0.)))

obj = Expr.add(Expr.dot(muDup, demand['demand (MW)'].values), 
               Expr.dot(muGup, supply['capacity (MW)'].values))
for n, n_node in enumerate(nodes):
    for m_node in bi_lines.xs(n_node).index:
        m = nodes.index(m_node)
        capacity = bi_lines.loc[(n_node, m_node), 'capacity (MW)']
        n_m_idx = bi_lines.index.get_loc((n_node, m_node))
        obj = Expr.add(obj, Expr.mul(capacity, 
                                     Expr.add(etaup.index(n_m_idx),
                                              etadown.index(n_m_idx))))
M.objective('obj', ObjectiveSense.Minimize, obj)
M.solve()

if M.getPrimalSolutionStatus() != SolutionStatus.Optimal or True:
    M.writeTask('proj1dual.opf')

demand['consumed (MW)'] = pd_const.dual()
supply['supplied (MW)'] = pg_const.dual()
buses = pd.DataFrame({
    'volt. angle (rad)': [theta.dual()[0] for theta in theta_eqs],
    'node price ($/MW)': lambdaN.level()
}, index=nodes)

In [14]:
summary = pd.concat([
    buses['node price ($/MW)'],
    demand.groupby('node')['consumed (MW)'].sum(),
    supply.groupby('node')['supplied (MW)'].sum(),
], axis=1).fillna(0.)
summary.columns = ['price ($/MW)', 'consumed (MW)', 'supplied (MW)']
display(summary)
print(summary.to_latex())

,price ($/MW),consumed (MW),supplied (MW)
Bus 1,20.0,0.0,100.0
Bus 2,20.0,100.0,50.0
Bus 3,20.0,50.0,0.0


\begin{tabular}{lrrr}
\toprule
{} &  price (\$/MW) &  consumed (MW) &  supplied (MW) \\
\midrule
Bus 1 &          20.0 &            0.0 &          100.0 \\
Bus 2 &          20.0 &          100.0 &           50.0 \\
Bus 3 &          20.0 &           50.0 &            0.0 \\
\bottomrule
\end{tabular}



In [15]:
print('Energy scheduled: {:g}'.format(demand['consumed (MW)'].sum()))
print('Objective value: {:g}'.format(M.primalObjValue()))
display(buses)
display(demand.sort_values('bid ($/MW)'))
display(supply.sort_values('offer ($/MW)'))

Energy scheduled: 150
Objective value: 3550


,volt. angle (rad),node price ($/MW)
Bus 1,0.0,20.0
Bus 2,-0.1,20.0
Bus 3,-0.1,20.0


,node,demand (MW),bid ($/MW),consumed (MW)
1,Bus 3,50.0,35.0,50.0
0,Bus 2,100.0,40.0,100.0


,node,capacity (MW),offer ($/MW),supplied (MW)
0,Bus 1,100.0,12.0,100.0
1,Bus 2,80.0,20.0,50.0


In [12]:
print(log.getvalue())

Problem
  Name                   : dual            
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 7               
  Cones                  : 0               
  Scalar variables       : 25              
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 3
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   : dual            
  Objective sens